In [1]:
#Imports, set cuda device, configure tensor board

import torch
from datasets import MovementDataset
from torchvision.transforms import v2, InterpolationMode
from modules.ego_motion_filter import EgoMotionFilter
from modules.identity import Identity
from torch import nn
import motion_trainer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

from torch.utils.tensorboard import SummaryWriter
import os
import shutil


TBOARD_LOGS = os.path.join(os.getcwd(), "tboard_logs", "vanilla_dae")
if not os.path.exists(TBOARD_LOGS):
    os.makedirs(TBOARD_LOGS)
shutil.rmtree(TBOARD_LOGS)
writer = SummaryWriter(TBOARD_LOGS)

cuda


In [2]:
root_dir = "/home/nfs/inf6/data/datasets/Carla_Moritz/SyncAngel3/"

transforms = v2.Compose([
    v2.Resize((256, 512), InterpolationMode.BILINEAR, antialias=False),
    v2.ToDtype(torch.float, scale=True),
])


train_dataset = MovementDataset(root_dir, transform=transforms, split='train')
valid_dataset = MovementDataset(root_dir, transform=transforms, split='validation')


train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=8)
valid_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=4, shuffle=True, num_workers=8)


filter = Identity()
model = EgoMotionFilter()
model.to(device)
model.eval()


EgoMotionFilter(
  (motion_estimator): MotionEstimator(
    (layer1): Sequential(
      (0): Conv2d(4096, 1024, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (layer2): Sequential(
      (0): Conv2d(1024, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (layer3): Sequential(
      (0): Conv2d(256, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (avg_pooling): AvgPool2d(kernel_size=(4, 8), stride=(4, 8), padding=0)
    (fc1): Linear(in_features=128, out_features=128, bias=True)
    (bn): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (camera_head): CameraHead(
    (fc1): Lin

In [3]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=10)

In [4]:
motion_trainer.train_model(model=model, optimizer=optimizer, scheduler=scheduler, criterion=criterion, 
                    train_loader=train_loader, valid_loader=valid_loader, num_epochs=2, writer=writer, device=device, save_frequency=1)

Epoch 1: loss 0.28383. :  11%|█         | 4689/42736 [19:33<2:38:38,  4.00it/s]  


KeyboardInterrupt: 